# 식시 - 양재역으로 검색
- 명칭, 주요 메뉴, 평점, 전화번호, 주소

In [1]:
import requests
import pandas as pd
from urllib.parse import quote

## 데이터 가져오기

In [2]:
base_url = 'https://www.siksinhot.com'
url = f'{base_url}/search?keywords={quote("양재역")}'
req = requests.get(url)
html = req.text
#html

## 찾으려는 데이터의 태그 찾기

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [4]:
lis = soup.select('div.listTy1 > ul > li')
len(lis)

12

In [5]:
li = lis[0]
href = li.select_one('a')['href']

In [6]:
url = base_url + href
req = requests.get(url)
sub_soop = BeautifulSoup(req.text,'html.parser')

In [7]:
store = sub_soop.select_one('.title')
info = store.find('h3').get_text()
info

'그릭슈바인 양재역점3.6HOT'

In [8]:
score = store.select_one('h3 > strong').string
score

'3.6'

In [9]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [16]:
menu = sub_soop.select('.store_info p')[1].get_text()
menu

'나이트라이프 > 맥주/호프'

In [11]:
tel = sub_soop.select_one('.p_tel p').get_text()
tel

'02-572-6100'

In [12]:
addr = sub_soop.select_one('.txt_adr').get_text()
addr

'서울특별시 서초구 강남대로 224'

In [19]:
name_list, score_list, menu_list,tel_list, addr_list = [],[],[],[],[]

for i, li in enumerate(lis):
    href = li.select_one('a')['href']
    url = base_url + href
    req = requests.get(url)
    sub_soop = BeautifulSoup(req.text,'html.parser')

    try:
        store = sub_soop.select_one('.title')
        info = store.find('h3').get_text()
        score = store.select_one('h3 > strong').string
        name = info[:info.find(score)]
        menu = sub_soop.select('.store_info p')[1].get_text()
        tel = sub_soop.select_one('.p_tel p').get_text()
        addr = sub_soop.select_one('.txt_adr').get_text()
    except:
        print(i)

    name_list.append(name)
    score_list.append(score)
    menu_list.append(menu)
    tel_list.append(tel)
    addr_list.append(addr)

3
4


In [26]:
df = pd.DataFrame({
    '업소명':name_list,'메뉴':menu_list,'평점':score_list,'번호':tel_list,'주소':addr_list
})
df

,업소명,메뉴,평점,번호,주소
0,그릭슈바인 양재역점,나이트라이프 > 맥주/호프,3.6,02-572-6100,서울특별시 서초구 강남대로 224
1,미스터브리즈 양재역점,세계음식 > 카페/커피숍,3.4,02-576-4333,서울 서초구 강남대로34길 6
2,동신참치 양재역점,한국음식 > 회,3.0,02-529-7130,서울 서초구 강남대로 213
3,동신참치 양재역점,한국음식 > 회,None,02-529-7130,서울 서초구 강남대로 213
4,동신참치 양재역점,한국음식 > 회,None,02-529-7130,서울 서초구 강남대로 213
5,영동족발 3호점,한국음식 > 족발/보쌈,4.1,02-575-0250,서울특별시 서초구 남부순환로 2628-18
6,황재벌,한국음식 > 장어구이/꼼장어,3.9,02-3471-5454,서울특별시 서초구 남부순환로347길 42-4 신화빌딩
7,김영모과자점 도곡타워점,세계음식 > 베이커리/제과점,4.1,02-3460-2005,서울특별시 강남구 언주로30길 10 현대비젼21
8,소호정 본점,한국음식 > 라면/칼국수/국수/수제비,4.0,02-579-7282,서울특별시 서초구 논현로 27
9,산동칼국수,한국음식 > 라면/칼국수/국수/수제비,3.9,02-3473-7972,서울특별시 서초구 강남대로37길 63


##  문제 발생한 두번째 업소 살펴보기

In [27]:
li = lis[3]
li.select_one('.cnt p').get_text

<bound method Tag.get_text of <p data-reactid="189"><!-- react-text: 190 -->딱새우회<!-- /react-text --><!-- react-text: 191 -->, 닭도리탕<!-- /react-text --><!-- react-text: 192 -->, 딱새우튀김<!-- /react-text --><!-- react-text: 193 -->, 술국닭계장<!-- /react-text --><!-- react-text: 194 -->, 마라해물새우<!-- /react-text --></p>>

In [ ]:
score = li.select_one('em').string
if score == '평가중':
    score = '0'

- 변경된 방법으로 재시도

In [32]:
name_list, score_list, menu_list,tel_list, addr_list = [],[],[],[],[]

for i, li in enumerate(lis):
    href = li.select_one('a')['href']
    url = base_url + href
    req = requests.get(url)
    sub_soop = BeautifulSoup(req.text,'html.parser')

    name = li.select_one('.store').string
    score = li.select_one('em').string
    if score == '평가중':
        score = '0'
    menu = li.select_one('.cnt p').get_text()

    try:
        tel = sub_soop.select_one('.p_tel p').get_text()
        addr = sub_soop.select_one('.txt_adr').get_text()
    
        name_list.append(name)
        score_list.append(score)
        menu_list.append(menu)
        tel_list.append(tel)
        addr_list.append(addr)
    except:
        print(i)


In [33]:
df = df = pd.DataFrame({
    '업소명':name_list,
    '메뉴':menu_list,
    '평점':score_list,
    '번호':tel_list,
    '주소':addr_list
})
df

,업소명,메뉴,평점,번호,주소
0,그릭슈바인 양재역점,"그릭슈바인bbq, 슈바이네 학센, 모듬 소세지 bbq, 콥 샐러드",3.6,02-572-6100,서울특별시 서초구 강남대로 224
1,미스터브리즈 양재역점,"아메리카노, 허니버터브레드, 와플",3.4,02-576-4333,서울 서초구 강남대로34길 6
2,동신참치 양재역점,"참치회, 참치정식",3.0,02-529-7130,서울 서초구 강남대로 213
3,새우닭 양재역점,"딱새우회, 닭도리탕, 딱새우튀김, 술국닭계장, 마라해물새우",0,02-575-6088,서울특별시 서초구 남부순환로356길 52 1층
4,양재역 옛날짜장,"짜장면, 짬뽕, 볶음밥, 고추잡채, 쟁반짜장, 탕수육",0,031-503-2550,경기도 시흥시 공단1대로 13
5,영동족발 3호점,"족발大, 쟁반막국수大, 순두부찌개, 족발 중, 해물파전",4.1,02-575-0250,서울특별시 서초구 남부순환로 2628-18
6,황재벌,"꼼장어, 쭈꾸미, 왕 꼼장어, 닭발, 계란찜, 알 주먹밥",3.9,02-3471-5454,서울특별시 서초구 남부순환로347길 42-4 신화빌딩
7,김영모과자점 도곡타워점,"몽블랑, 바게트샌드위치, 레즌 천연발효빵, 생크림앙팡, 마늘바게트",4.1,02-3460-2005,서울특별시 강남구 언주로30길 10 현대비젼21
8,소호정 본점,"안동국시, 국밥, 수육(대), 전(대), 묵(대)",4.0,02-579-7282,서울특별시 서초구 논현로 27
9,산동칼국수,"손칼국수, 만두국, 왕만두",3.9,02-3473-7972,서울특별시 서초구 강남대로37길 63
